**ASSIGNMENT 4**

Group Name: 61

Member Names: Owen Jory

Member Student Numbers: 300168367

**Derived Datasets**

This notebook is a starting point for Assignment 4. In this assignment, you will perform a classification empirical study. This notebook will help you to create derived datasets in Section 2 of the assignment.

In [1]:
#let's start by installing spaCy
!pip install spacy
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.1/12.8 MB 2.6 MB/s eta 0:00:05
     - -------------------------------------- 0.3/12.8 MB 4.2 MB/s eta 0:00:03
     -- ------------------------------------- 0.7/12.8 MB 5.2 MB/s eta 0:00:03
     --- ------------------------------------ 1.0/12.8 MB 6.0 MB/s eta 0:00:02
     ---- ----------------------------------- 1.3/12.8 MB 5.9 MB/s eta 0:00:02
     ---- ----------------------------------- 1.6/12.8 MB 6.2 MB/s eta 0:00:02
     ----- ---------------------------------- 1.9/12.8 MB 6.0 MB/s eta 0:00:02
     ------- -------------------------------- 2.3/12.8 MB 6.3 MB/s eta 0:00:02
     ------- -------------------------------- 2.4/12.8 MB 6.3 MB/s eta 0:00:02
     -------- ------------------------------- 2.8/12.8 MB 6.1 MB/s eta 0:00:02
     ---------- ----------------------------- 3.2/12.8 MB 6.5 MB/s eta 0:00:02
     ----------- ---------------------------- 3.7/12.8 MB 6

In [2]:
import spacy
import pandas as pd
import numpy as np

In [30]:
from sklearn.linear_model import LogisticRegression as LR
from sklearn.neural_network import MLPClassifier as MLP
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing

You have been given a list of datasets in the assignment description. Choose one of the datasets and provide the link below and read the dataset using pandas. You should provide a link to your own Github repository even if you are using a reduced version of a dataset from your TA's repository.

add description of the dataset and your justification of the choices made to obtain the derived datasets

In [4]:
#Load the dataset you chose.
# Make sure the Notebook can load your dataset, just like previous assignments.

url = 'https://raw.githubusercontent.com/SmolTortol/CSI4106_a4/main/AirlinePassengerReviews.csv'

#provide the link to the raw version of dataset. You *need* to provide a link to *your own* github repository. DO NOT use the link that is provided as an example.

In [5]:
print(url)
data = pd.read_csv(url, on_bad_lines='skip')

https://raw.githubusercontent.com/SmolTortol/CSI4106_a4/main/AirlinePassengerReviews.csv


In [6]:
data.head()

,customer_review,NPS Score
0,London to Izmir via Istanbul. First time I'd ...,Passive
1,Istanbul to Bucharest. We make our check in i...,Detractor
2,Rome to Prishtina via Istanbul. I flew with t...,Detractor
3,Flew on Turkish Airlines IAD-IST-KHI and retu...,Promoter
4,Mumbai to Dublin via Istanbul. Never book Tur...,Detractor


This is where you create the NLP pipeline. load() will download the correct model (English).

In [7]:
nlp = spacy.load("en_core_web_sm")

Applying the pipeline to every sentences creates a Document where every word is a Token object.

Doc: https://spacy.io/api/doc

Token: https://spacy.io/api/token

In [8]:
#Apply nlp pipeline to the column that has your sentences.
data['tokenized'] = data['customer_review'].apply(nlp)

In [9]:
data.head()

,customer_review,NPS Score,tokenized
0,London to Izmir via Istanbul. First time I'd ...,Passive,"( , London, to, Izmir, via, Istanbul, ., First..."
1,Istanbul to Bucharest. We make our check in i...,Detractor,"( , Istanbul, to, Bucharest, ., We, make, our,..."
2,Rome to Prishtina via Istanbul. I flew with t...,Detractor,"( , Rome, to, Prishtina, via, Istanbul, ., I, ..."
3,Flew on Turkish Airlines IAD-IST-KHI and retu...,Promoter,"( , Flew, on, Turkish, Airlines, IAD, -, IST, ..."
4,Mumbai to Dublin via Istanbul. Never book Tur...,Detractor,"( , Mumbai, to, Dublin, via, Istanbul, ., Neve..."


A Token object has many attributes such as part-of-speech (pos_), lemma (lemma_), etc. Take a look at the documentation to see all attributes.

The following function is an example on how you can fetch a specific pos tagging from a sentence. We return the lemmatization because we only want the infinitive word.

In [10]:
#create empty dataframes that will store your derived datasets

derived_dataset1 = pd.DataFrame(columns = ['Class', 'pos'])
derived_dataset2 = pd.DataFrame(columns = ['Class', 'pos-np'])

In [11]:
def get_pos(sentence, wanted_pos=[], wanted_ents=[]): #wanted_pos refers to the desired pos tagging
    verbs = []
    for token in sentence:
        if token.pos_ in wanted_pos:
            verbs.append(token.lemma_) # lemma returns a number. lemma_ return a string
    for ent in sentence.ents:
        if ent.label_ in wanted_ents:
            verbs.append(ent.text)
    return ' '.join(verbs) # return value is as a string and not a list for countVectorizer

In [12]:
#As an example, we use the above function to fetch all the verbs. We store this information in our first derived dataset
derived_dataset1['pos'] = data['tokenized'].apply(lambda sent : get_pos(sent, ['ADJ']))

In [13]:
derived_dataset1.head()

,Class,pos
0,NaN,first good nice great Most contradictory littl...
1,NaN,first last
2,NaN,several past bad bad normal most useless few w...
3,NaN,excellent inflight extensive easy excellent in...
4,NaN,turkish other more


In [14]:
#Change this line to fetch your desired pos taggings for the second derived dataset
derived_dataset2['pos-np'] = data['tokenized'].apply(lambda sent : get_pos(sent, ['VERB'], ['ORG', 'MONEY']))

In [15]:
derived_dataset2.head()

,Class,pos-np
0,NaN,fly find delay miss speak give speak get make ...
1,NaN,make take go let ™ say overbooke have wait bring
2,NaN,fly say get fly have try ask connect relax lan...
3,NaN,return maintain fly leave use seem have show h...
4,NaN,travel get have ask stay care make keep wait


Now that you have your derived datasets, you can move to perform your classificaton task.

**Perform Classification Task**

In [18]:
count_vect = CountVectorizer()
default_vect_dataset = count_vect.fit_transform(data['customer_review'])
derived_vect_dataset1 = count_vect.fit_transform(derived_dataset1['pos'])
derived_vect_dataset2 = count_vect.fit_transform(derived_dataset2['pos-np'])

Base Dataset

In [58]:
logreg = LR()
mlp = MLP(hidden_layer_sizes=(10,))
mlp_split_hidden = MLP(hidden_layer_sizes=(5, 5))
mlp_adaptive = MLP(hidden_layer_sizes=(10,), learning_rate='adaptive')

In [59]:
features_train, features_test, target_train, target_test = train_test_split(default_vect_dataset, data['NPS Score'], test_size=1/4)

In [60]:
logreg.fit(features_train, target_train)
target_pred = logreg.predict(features_test)
print(metrics.accuracy_score(target_pred, target_test))

0.8166822867853796


C:\Users\owenj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [61]:
mlp.fit(features_train, target_train)
target_pred = mlp.predict(features_test)
print(metrics.accuracy_score(target_pred, target_test))

0.7904404873477039


In [62]:
mlp_split_hidden.fit(features_train, target_train)
target_pred = mlp_split_hidden.predict(features_test)
print(metrics.accuracy_score(target_pred, target_test))

0.7811308965948142


In [63]:
mlp_adaptive.fit(features_train, target_train)
target_pred = mlp_adaptive.predict(features_test)
print(metrics.accuracy_score(target_pred, target_test))

0.7875663855045298


Adjective Dataset

In [85]:
logreg = LR()
mlp = MLP(hidden_layer_sizes=(10,))
mlp_split_hidden = MLP(hidden_layer_sizes=(5, 5))
mlp_adaptive = MLP(hidden_layer_sizes=(10,), learning_rate='adaptive')

In [86]:
features_train, features_test, target_train, target_test = train_test_split(derived_vect_dataset1, data['NPS Score'], test_size=1/4)

In [94]:
logreg.fit(features_train, target_train)
target_pred = logreg.predict(features_test)
print(metrics.accuracy_score(target_pred, target_test))

0.7007810059356451


C:\Users\owenj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [97]:
mlp.fit(features_train, target_train)
target_pred = mlp.predict(features_test)
print(metrics.accuracy_score(target_pred, target_test))

0.662293033427054


C:\Users\owenj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [95]:
mlp_split_hidden.fit(features_train, target_train)
target_pred = mlp_split_hidden.predict(features_test)
print(metrics.accuracy_score(target_pred, target_test))

0.6609184629803186


C:\Users\owenj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [96]:
mlp_adaptive.fit(features_train, target_train)
target_pred = mlp_adaptive.predict(features_test)
print(metrics.accuracy_score(target_pred, target_test))

0.6539831302717901


C:\Users\owenj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Named Entities

In [98]:
logreg = LR()
mlp = MLP(hidden_layer_sizes=(10,))
mlp_split_hidden = MLP(hidden_layer_sizes=(5, 5))
mlp_adaptive = MLP(hidden_layer_sizes=(10,), learning_rate='adaptive')

In [99]:
features_train, features_test, target_train, target_test = train_test_split(derived_vect_dataset2, data['NPS Score'], test_size=1/4)

In [100]:
logreg.fit(features_train, target_train)
target_pred = logreg.predict(features_test)
print(metrics.accuracy_score(target_pred, target_test))

0.7072789753202124


C:\Users\owenj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [101]:
mlp.fit(features_train, target_train)
target_pred = mlp.predict(features_test)
print(metrics.accuracy_score(target_pred, target_test))

0.6621055920024992


C:\Users\owenj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [102]:
mlp_split_hidden.fit(features_train, target_train)
target_pred = mlp_split_hidden.predict(features_test)
print(metrics.accuracy_score(target_pred, target_test))

0.6715401437050922


C:\Users\owenj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [103]:
mlp_adaptive.fit(features_train, target_train)
target_pred = mlp_adaptive.predict(features_test)
print(metrics.accuracy_score(target_pred, target_test))

0.6629803186504217


C:\Users\owenj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


**Analysis**

|Model|Raw Data Accuracy|Adjective Dataset Accuracy|Named Entities Dataset Accuracy|
|---|---|---|---|
|Logistic Regression Model|81.7|70.1|70.7|
|Multilayer Perceptron Model|79.0|66.2|66.2|
|MLP with 2 Layers|78.1|66.1|67.2|
|MLP with Adaptive Learning|78.8|65.4|66.3|

In all cases, the logistic regression model outperformed the multilayer perceptron model. Though, I suspect this may be because I had to reduce the size of the hidden layers for performance reasons*. Neither dividing the nodes within the perceptron model nor changing the learning mode to 'adaptive' had any major impact on accuracy. 

*Performance issues include: taking 3 hours to excecute 1 line of code, randomly crashing jupyter notebook, and randomly deleting the previous few hours of work... multiple times